# 최단 경로 알고리즘

그래프에서 두 정점 사이의 가장 짧은 경로를 찾는 알고리즘이다.

- Dijkstra : 한 정점 → 모든 정점 최단 거리
- Bellman-Ford : 한 정점 → 모든 정점 + 음수 처리
- Floyd-Warshall : 모든 정점 ↔ 모든 정점 최단 거리 
- 0-1 BFS : 가중치가 0 or 1인 그래프에서 최단 거리

In [1]:
graph = {
  0: [(1, 2), (2, 4)],
  1: [(2, 1), (3, 7)],
  2: [(3, 3)],
  3: []
}
edges = [(u, v, w) for u in graph for v, w in graph[u]]
V = 4  # 정점 수
start = 0

In [2]:
# 최단 경로 복원 (0-1 BFS, Dijkstra, Bellman-Ford)
def reconstruct_path(prev, start, end):
  path = []
  # end 노드부터 prev 배열을 따라가며 경로 추척
  while end != -1:
    path.append(end)
    end = prev[end] # 이전 노드로 이동

  # 경로의 시작점이 start와 일치할 경우 → 정상 경로, 아니면 빈 리스트 반환
  return path[::-1] if path[-1] == start else []


In [3]:
# 0-1 BFS: 가중치가 0 또는 1인 그래프에서 최단 거리 탐색
from collections import deque

def zero_one_bfs(start, graph, V):
  # 최단 거리 배열 및 경로 추적 배열 초기화
  dist = [float('inf') for _ in range(V)]
  dist[start] = 0
  prev = [-1 for _ in range(V)]

  # BFS 탐색
  queue = deque([start])
  while queue:
    node = queue.popleft()

    # 현재 노드와 연결된 모든 노드 확인 (최단거리 확인 시 갱신)
    for next, cost in graph[node]:
      if dist[next] > dist[node] + cost:
        dist[next] = dist[node] + cost
        prev[next] = node

        if cost == 0:
          queue.appendleft(next)  # 가중치 0 → 우선 탐색
        else:
          queue.append(next)      # 가중치 1 → 나중 탐색

  return dist, prev

dist, prev = zero_one_bfs(start, graph, V)
print("0-1 BFS 경로:", reconstruct_path(prev, start, 3))
print("0-1 BFS 거리:", dist)

0-1 BFS 경로: [0, 1, 2, 3]
0-1 BFS 거리: [0, 2, 3, 6]


In [4]:
# Dijkstra : 한 정점에서 다른 정점들의 최단 거리 탐색 (양의 그래프만)
import heapq

def dijkstra(start, graph, V):
  # 최단 거리 배열 및 경로 추적 배열 초기화
  dist = [float('inf') for _ in range(V)]
  dist[start] = 0
  prev = [-1 for _ in range(V)]

  priority_queue = [(0, start)]
  while priority_queue:
    cost, node = heapq.heappop(priority_queue)

    # 이미 저장된 거리보다 긴 거리는 무시
    if cost > dist[node]:
      continue

    # 현재 정점 u에서 인접한 모든 정점 v 확인 (최단거리 확인 시 갱신)
    for next, weight in graph[node]:
      if dist[next] > cost + weight:
        dist[next] = cost + weight
        prev[next] = node
        heapq.heappush(priority_queue, (dist[next], next))
  
  return dist, prev
  
dist, prev = dijkstra(start, graph, V)
print("Dijkstra 경로:", reconstruct_path(prev, start, 3))
print("Dijkstra 거리:", dist)

Dijkstra 경로: [0, 1, 2, 3]
Dijkstra 거리: [0, 2, 3, 6]


In [5]:
# Bellman-Ford : 한 정점에서 다른 정점들의 최단 거리 탐색 (음의 그래프 허용)
def bellman_ford(start, edges, V):
  # 최단 거리 배열 초기화
  dist = [float('inf') for _ in range(V)]
  dist[start] = 0

  prev = [-1 for _ in range(V)]

  # 전체 간선을 순회하면 거리 갱신
  for _ in range(V-1):
    for node, next, cost in edges:
      # 시작점에 도달 가능한 노드에서만 갱신 시도
      if dist[node] != float('inf') and dist[next] > dist[node] + cost:
        dist[next] = dist[node] + cost
        prev[next] = node

  # 음수 사이클 검사
  for node, next, cost in edges:
    if dist[next] > dist[node] + cost:
      return 'NEGATIVE CYCLE'
    
  return dist, prev

dist, prev = bellman_ford(start, edges, V)
print("Bellman-Ford 경로:", reconstruct_path(prev, start, 3))
print("Bellman-Ford 거리:", dist)

Bellman-Ford 경로: [0, 1, 2, 3]
Bellman-Ford 거리: [0, 2, 3, 6]


In [6]:
# 최단 경로 복원 (Floyd-Warshall)
def reconstruct_fw_path(nxt, u, v):
  # 경로가 존재하지 않으면 빈 리스트 반환
  if nxt[u][v] == -1:
    return []
  
  # 경로 추적 (도착 노드까지 이동)
  path = [u]
  while u != v:
    u = nxt[u][v]
    path.append(u)
  return path

In [7]:
# Floyd-Warshall : 모든 정점 간의 모든 경로 탐색
def floyd_warshall(graph, V):
  # 최단 거리 및 경로 복원 배열 초기화
  dist = [[float('inf') for _ in range(V)] for _ in range(V)]
  next_node = [[-1 for _ in range(V)] for _ in range(V)]
  
  
  # 그래프 간선 정보로 초기 거리 및 경로 설정
  for node in range(V):
    dist[node][node] = 0
    for next, cost in graph[node]:
      dist[node][next] = cost
      next_node[node][next] = next

  # 경유지 k를 통해 i → j 거리 갱신 여부 판단
  for k in range(V):
    for i in range(V):
      for j in range(V):
        if dist[i][k] + dist[k][j] < dist[i][j]:
          dist[i][j] = dist[i][k] + dist[k][j]
          next_node[i][j] = next_node[i][k]


  return dist, next_node

dist, nxt = floyd_warshall(graph, V)
print("Floyd-Warshall 경로 (0→3):", reconstruct_fw_path(nxt, 0, 3))
print("Floyd-Warshall 거리 (0→3):", dist[0][3])

Floyd-Warshall 경로 (0→3): [0, 1, 2, 3]
Floyd-Warshall 거리 (0→3): 6
